# Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Importing the dataset

In [ ]:
df = pd.read_csv('Transaction.csv')
df.sample
df.head(10)

,Trans_Id,Items
0,1,"F,G,I,J"
1,2,"G,H,J"
2,3,"G,I,J"
3,4,"G,H,J"
4,5,"G,H,I,J"
5,6,"G,H,I"


# Pre-processing

In [ ]:
all_items = sorted(set(item.strip() for sublist in df['Items'].str.split(',') for item in sublist if item))
all_items

['F', 'G', 'H', 'I', 'J']

In [ ]:
transaction_df = pd.DataFrame(columns=['Trans_Id'] + all_items)
transaction_df['Trans_Id'] = df['Trans_Id']
transaction_df

,Trans_Id,F,G,H,I,J
0,1,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN
5,6,NaN,NaN,NaN,NaN,NaN


In [ ]:
for index, row in df.iterrows():
    items = set(row['Items'].split(','))
    for item in all_items:
        transaction_df.at[index, item] = True if item in items else False
transaction_df.head()

,Trans_Id,F,G,H,I,J
0,1,True,True,False,True,True
1,2,False,True,True,False,True
2,3,False,True,False,True,True
3,4,False,True,True,False,True
4,5,False,True,True,True,True


In [ ]:
transaction_df.drop('Trans_Id', axis=1, inplace=True)
transaction_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,F,G,H,I,J
0,True,True,False,True,True
1,False,True,True,False,True
2,False,True,False,True,True
3,False,True,True,False,True
4,False,True,True,True,True
5,False,True,True,True,False


# Construct model using Association Rules

## Apply FP-growth to determine all the frequent itemsets. Set minimum support value to 50%

In [ ]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(transaction_df, min_support=0.50, use_colnames=True)
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,1.0,(G)
1,0.666667,(H)
2,0.666667,(I)
3,0.833333,(J)
4,0.666667,"(G, H)"
5,0.666667,"(I, G)"
6,0.833333,"(G, J)"
7,0.5,"(J, H)"
8,0.5,"(I, J)"
9,0.5,"(G, J, H)"


## Apply “Create Association Rules” operators. Set confidence value to 75%

In [ ]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.75)
rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(G),(H),1.000000,0.666667,0.666667,0.666667,1.0,0.000000,1.000000,0.00
1,(H),(G),0.666667,1.000000,0.666667,1.000000,1.0,0.000000,inf,0.00
2,(I),(G),0.666667,1.000000,0.666667,1.000000,1.0,0.000000,inf,0.00
3,(G),(I),1.000000,0.666667,0.666667,0.666667,1.0,0.000000,1.000000,0.00
4,(G),(J),1.000000,0.833333,0.833333,0.833333,1.0,0.000000,1.000000,0.00
5,(J),(G),0.833333,1.000000,0.833333,1.000000,1.0,0.000000,inf,0.00
6,(J),(H),0.833333,0.666667,0.500000,0.600000,0.9,-0.055556,0.833333,-0.40
7,(H),(J),0.666667,0.833333,0.500000,0.750000,0.9,-0.055556,0.666667,-0.25
8,(I),(J),0.666667,0.833333,0.500000,0.750000,0.9,-0.055556,0.666667,-0.25
9,(J),(I),0.833333,0.666667,0.500000,0.600000,0.9,-0.055556,0.833333,-0.40


# Model deployment using Association Rules

## List all the frequent itemsets that have been found (with their support)

In [ ]:
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,1.0,(G)
1,0.666667,(H)
2,0.666667,(I)
3,0.833333,(J)
4,0.666667,"(G, H)"
5,0.666667,"(I, G)"
6,0.833333,"(G, J)"
7,0.5,"(J, H)"
8,0.5,"(I, J)"
9,0.5,"(G, J, H)"


## What is the maximal frequent itemset found? Give its support? List all the strong association rules that can be generated from it.

In [ ]:
rules.sort_values(by='confidence', ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
12,"(H, J)",(G),0.500000,1.000000,0.500000,1.000000,1.0,0.000000,inf,0.00
2,(I),(G),0.666667,1.000000,0.666667,1.000000,1.0,0.000000,inf,0.00
5,(J),(G),0.833333,1.000000,0.833333,1.000000,1.0,0.000000,inf,0.00
17,"(I, J)",(G),0.500000,1.000000,0.500000,1.000000,1.0,0.000000,inf,0.00
1,(H),(G),0.666667,1.000000,0.666667,1.000000,1.0,0.000000,inf,0.00
4,(G),(J),1.000000,0.833333,0.833333,0.833333,1.0,0.000000,1.000000,0.00
11,"(G, H)",(J),0.666667,0.833333,0.500000,0.750000,0.9,-0.055556,0.666667,-0.25
19,(I),"(G, J)",0.666667,0.833333,0.500000,0.750000,0.9,-0.055556,0.666667,-0.25
7,(H),(J),0.666667,0.833333,0.500000,0.750000,0.9,-0.055556,0.666667,-0.25
8,(I),(J),0.666667,0.833333,0.500000,0.750000,0.9,-0.055556,0.666667,-0.25
